In [ ]:
import uproot
import glob
import numpy as np
import pandas as pd
from scipy.constants import epsilon_0, e as q_e
from scipy.interpolate import griddata


from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
from matplotlib.colorbar import ColorbarBase
from matplotlib.colors import Normalize
import matplotlib.colors as mcolors
from matplotlib.collections import LineCollection
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

import trimesh
from trimesh.points import PointCloud

from common_functions import *

In [ ]:
stacked_spheres = trimesh.load_mesh('../sphere-charging/geometry/stacked_spheres_frompython.stl') 

# get a single cross section of the mesh
section = stacked_spheres.section(plane_origin=stacked_spheres.centroid, plane_normal=[0, 1, 0])
slice_2D, to_3D = section.to_2D()
slice_2D.show()

In [ ]:
# test
configIN = "onlysolarwind"
filenames = sorted(glob.glob(f"../build-10um-uniform/fieldmaps/*107*{configIN}*.txt"))
print(filenames)

# Usage
df_uniform_iteration1  = read_uniform_fieldmap(filenames[0])
if "E_mag" not in df_uniform_iteration1.columns:
    df_uniform_iteration1["E_mag"] = np.linalg.norm(df_uniform_iteration1[["Ex", "Ey", "Ez"]].values, axis=1)

max(df_uniform_iteration1["E_mag"])

In [ ]:
# test
configIN = "onlysolarwind"
filenames = sorted(glob.glob(f"../build-adaptive-mesh/fieldmaps/*113*{configIN}*.txt"))

# Usage
df_adaptive_iteration1, metadata  = read_adaptive_fieldmap(filenames[0])
if "E_mag" not in df_adaptive_iteration1.columns:
    df_adaptive_iteration1["E_mag"] = np.linalg.norm(df_adaptive_iteration1[["Ex", "Ey", "Ez"]].values, axis=1)

max(df_adaptive_iteration1["E_mag"])

In [ ]:
# plot the distribution of the field to get a sense for the overall field values before analyzing the field in detail
plt.hist(df_uniform_iteration1[df_uniform_iteration1["E_mag"]>0]["E_mag"],bins=np.logspace(0,8,100),alpha=0.8,label="Uniform - 10 um")
plt.hist(df_adaptive_iteration1[df_adaptive_iteration1["E_mag"]>0]["E_mag"],bins=np.logspace(0,8,100),alpha=0.2,label="Adaptive Mesh")
plt.xscale("log")
plt.yscale("log")
plt.xlabel("|E| (V/m)")
plt.ylabel("Counts")
plt.title("Iteration 1")
plt.legend()
plt.show()

In [ ]:
len(df_adaptive_iteration1[df_adaptive_iteration1["E_mag"]>1e3]["E_mag"])

In [ ]:
max(df_uniform_iteration1["E_mag"])

In [ ]:
stacked_spheres_cropped = stacked_spheres.copy()

bbox_min = np.array([-0.2, -0.3, 0])
bbox_max = np.array([ 0.2,  0.1,  100])

# Filter vertices
in_box = np.all((stacked_spheres_cropped.vertices >= bbox_min) & 
                (stacked_spheres_cropped.vertices <= bbox_max), axis=1)

# Get face indices where all 3 vertices are in the box
face_mask = np.all(in_box[stacked_spheres_cropped.faces], axis=1)

# Extract submesh
stacked_spheres_cropped = stacked_spheres_cropped.submesh([face_mask], only_watertight=False, append=True)

# Usage with black semi-transparent edges
scene = plot_trimesh_edges_only(stacked_spheres_cropped, edge_color=[0, 0, 0, 350])  # Black, 50% transparent

# Create a PointCloud object
#gamma_photoemission_sites = trimesh.points.PointCloud(np.array(all_gamma_holes_df["Pre_Step_Position_mm"].tolist()), colors=[0, 255, 0, 255])  # RGBA green points
iterationIN = 2
fieldIN = df_adaptive_iteration1
threshold = 1e2
largefieldpoints = np.array(fieldIN[fieldIN["E_mag"] > threshold][["x", "y", "z"]].to_numpy().tolist())

In [ ]:

# fieldpoints = []

# for pointIN in largefieldpoints:
#     sphere = trimesh.creation.icosphere(radius=0.005)  # Create a sphere for each point
    
#     # Apply translation to the sphere (move it to the position of the point)
#     sphere.apply_translation(np.array(pointIN))
    
#     # Set the sphere color to red
#     sphere.visual.face_colors = [255, 0, 0, 255]  # red spheres
    
#     # Append the sphere to the list
#     fieldpoints.append(sphere)

# # Add all the spheres to the scene
# scene.add_geometry(fieldpoints)

max_arrows = 3000
field_largevalues = fieldIN[fieldIN["E_mag"] > threshold]
field_plot = field_largevalues.sample(n=max_arrows, random_state=42) if len(field_largevalues) > max_arrows else field_largevalues

directions = field_plot[['Ex', 'Ey', 'Ez']].values
magnitudes = np.linalg.norm(directions, axis=1)
directions_unit = np.where(magnitudes[:, None] > 0, directions / magnitudes[:, None], 0)

# Avoid log(0) by adding a small value (e.g., 1e-12) if needed
log_magnitudes = np.log10(magnitudes + 1e-12)  # Log base 10 normalization

# Normalize log-magnitudes to [0, 1]
norm_magnitudes = (log_magnitudes - log_magnitudes.min()) / (np.ptp(log_magnitudes) + 1e-12)

# Create a colormap
cmap = plt.cm.jet
colors_rgba = cmap(norm_magnitudes)  # shape: (N, 4) with floats in [0, 1]

# Convert to 0–255 uint8 for trimesh
colors_rgba = (colors_rgba * 255).astype(np.uint8)

# Define base dimensions
base_arrow_length = 0.05  # This will be scaled by magnitude
arrow_radius = 0.001
cone_ratio = 0.2  # Cone length as fraction of total arrow length

# Normalize magnitudes for scaling arrow lengths
scaling_factor = 0.5  # Adjust this to control overall arrow size
scaled_lengths = base_arrow_length * scaling_factor * norm_magnitudes

for pos, dir_vec, color, magnitude_norm, scaled_length in zip(
    field_plot[['x', 'y', 'z']].values, 
    directions_unit, 
    colors_rgba, 
    norm_magnitudes,
    scaled_lengths
):
    if np.linalg.norm(dir_vec) == 0:
        continue

    # Calculate arrow dimensions based on scaled length
    arrow_length = scaled_length
    cone_length = arrow_length * cone_ratio
    shaft_length = arrow_length - cone_length

    # Skip very small arrows to avoid visualization issues
    # if arrow_length < 1e-4:
    #     continue

    # Create shaft - centered at origin along Z-axis
    shaft = trimesh.creation.cylinder(radius=arrow_radius, height=shaft_length, sections=12)
    shaft.apply_translation([0, 0, shaft_length / 2])  # Move so base is at z=0, top at z=shaft_length

    # Create cone - positioned to touch the shaft
    cone = trimesh.creation.cone(radius=arrow_radius * 2, height=cone_length, sections=12)
    # Position cone so its base touches the top of the shaft (at z=shaft_length)
    cone.apply_translation([0, 0, shaft_length])

    # Combine parts
    arrow = trimesh.util.concatenate([shaft, cone])

    # Set color per face
    arrow.visual.face_colors = np.tile(color, (arrow.faces.shape[0], 1))

    # Align +Z to direction
    transform = trimesh.geometry.align_vectors([0, 0, 1], dir_vec)
    transform[:3, 3] = pos
    arrow.apply_transform(transform)

    scene.add_geometry(arrow)

scene.show()

In [ ]:
## determine appropiate grid spacing for Electric Field calculations

# Flatten the mesh grid to create sampling points
sampling_points = np.column_stack([np.array(df["x"]),np.array(df["y"]),np.array(df["z"])])
print(len(sampling_points))
adaptive_grid_points = trimesh.points.PointCloud(sampling_points, colors=[0, 255, 0, 255])

scene = plot_trimesh_edges_only(stacked_spheres, edge_color=[0, 0, 0, 128])
scene.add_geometry([adaptive_grid_points])
scene.show()

In [ ]:
fileIN

## Case 1: SW electrons and ions

In [ ]:
directory_path = "../build-adaptive-mesh/root/"
configIN = "solarwind"
filelist = sorted(glob.glob(f"{directory_path}/*stackediteration*{configIN}*num100000.root"))

all_incident_protons_inside, all_incident_electrons_inside = [],[]

for fileIN in filelist:

    print(fileIN.split("/")[-1])
    number_str = fileIN.split("/")[-1].split("_")[1]
    iterationNUM = int(''.join(filter(str.isdigit, number_str)))

    if iterationNUM > 28:
        break

    # read data from different iterations
    vars()["protons_inside_"+str(number_str)], vars()["electrons_inside_"+str(number_str)] = calculate_stats(read_rootfile(fileIN.split("/")[-1], directory_path=directory_path), 
                                                                                                             config=configIN)
    all_incident_protons_inside.append(vars()["protons_inside_"+str(number_str)])
    all_incident_electrons_inside.append(vars()["electrons_inside_"+str(number_str)])
    print(78*"-")

# Concatenate all iterations into single DataFrames
all_incident_protons_inside_df = pd.concat(all_incident_protons_inside, ignore_index=True)
all_incident_electrons_inside_df = pd.concat(all_incident_electrons_inside, ignore_index=True)

In [ ]:
# Concatenate all iterations into single DataFrames
all_incident_protons_inside_df = pd.concat(all_incident_protons_inside, ignore_index=True)
all_incident_electrons_inside_df = pd.concat(all_incident_electrons_inside, ignore_index=True)

In [ ]:
protons_inside_stackediteration0

In [ ]:
surf, ilm_values = plot_face_illumination(all_incident_protons_inside_df, stacked_spheres, vmin=0, vmax=1)

In [ ]:
plt.hist(ilm_values, bins=np.logspace(-1,3,100))
plt.xscale("log")
plt.title("# of Photons hitting each Voxel")
plt.show()

print(f"Distribution Stats: mean {np.mean(ilm_values[ilm_values!=0])}, median {np.median(ilm_values[ilm_values!=0])}, max {np.max(ilm_values)}")

# our area is a factor of 4 smaller than their area 
print(f"for one iteration, median # of particles in each equivalently sized voxel is {np.median(ilm_values[ilm_values!=0])/4}")
voxel_area = 0.0004/(1000)**2 # rough area approximated from python (0.4 micron2)
zimmerman_charge = 1*(1e-6) # C/m2
zimmerman_electronnum = (zimmerman_charge/1.60217663e-19)*voxel_area
print(f"will take {zimmerman_electronnum/(np.max(ilm_values[ilm_values!=0])*4)} iterations at this rate to get to the photoemission flux ranges shown at 3 seconds")

In [ ]:
iterationIN=0
surface,facolors = plot_surface_potential_fornegativepositive_charge(all_incident_electrons_inside_df, all_incident_protons_inside_df, stacked_spheres, vmin=-10,vmax=10)

plt.hist(facolors[facolors!=0],bins=50)
plt.show()

# Make sure each triangle has its own unique vertices
surface_edited = surface.copy()
surface_edited.unmerge_vertices()
surface_edited.visual.vertex_colors = None
surface_edited.show()

In [ ]:
31919

In [ ]:
# Define colormap and normalization
cmap = plt.cm.seismic
norm = Normalize(vmin=-0.2, vmax=0.2)

# Create a figure and a single axis for the colorbar
fig, ax = plt.subplots(figsize=(6, 1))
fig.subplots_adjust(bottom=0.5)

# Create the colorbar
cb = ColorbarBase(ax, cmap=cmap, norm=norm, orientation='horizontal')
cb.set_label('Surface Potential (mV)')  # Optional label

plt.show()


In [ ]:
bbox_min = np.array([-0.25, -0.31, 0])
bbox_max = np.array([ 0.15,  0.1,  100])

# Filter vertices
in_box = np.all((surface_edited.vertices >= bbox_min) & 
                (surface_edited.vertices <= bbox_max), axis=1)

# Get face indices where all 3 vertices are in the box
face_mask = np.all(in_box[surface_edited.faces], axis=1)

# Extract submesh
cropped = surface_edited.submesh([face_mask], only_watertight=False, append=True)
cropped.show()

## Case 2: Photoemission (incident gammas)

In [ ]:
configIN = "onlyphotoemission"
directory_path = "../build-adaptive-mesh/root/"
filelist = sorted(glob.glob(f"{directory_path}/*stackediteration*{configIN}*num100000.root"))

all_gamma_holes = []
all_electrons_inside = []

for fileIN in filelist:
    print(fileIN.split("/")[-1])
    number_str = fileIN.split("/")[-1].split("_")[1]
    iterationNUM = int(''.join(filter(str.isdigit, number_str)))

    # read data from different iterations
    vars()["gamma_holes_"+str(number_str)], _, vars()["electrons_inside_"+str(number_str)] = calculate_stats(read_rootfile(fileIN.split("/")[-1], directory_path=directory_path),
                                                                                                             config=configIN)
    all_gamma_holes.append(vars()["gamma_holes_"+str(number_str)])
    all_electrons_inside.append(vars()["electrons_inside_"+str(number_str)])
    print(78*"-")

# Concatenate all iterations into single DataFrames
all_gamma_holes_df = pd.concat(all_gamma_holes, ignore_index=True)
all_electrons_inside_df = pd.concat(all_electrons_inside, ignore_index=True)

In [ ]:
gamma_holes_stackediteration0

In [ ]:
surf, ilm_values = plot_face_illumination(gamma_holes_stackediteration0, stacked_spheres, vmin=0, vmax=1)

In [ ]:
plt.hist(ilm_values, bins=np.logspace(-1,3,100))
plt.xscale("log")
plt.title("# of Photons hitting each Voxel")
plt.show()

print(f"Distribution Stats: mean {np.mean(ilm_values[ilm_values!=0])}, median {np.median(ilm_values[ilm_values!=0])}, max {np.max(ilm_values)}")

# our area is a factor of 4 smaller than their area 
print(f"for one iteration, median # of particles in each equivalently sized voxel is {np.median(ilm_values[ilm_values!=0])/4}")
voxel_area = 0.0004/(1000)**2 # rough area approximated from python (0.4 micron2)
zimmerman_charge = 0.5*(1e-6) # C/m2
zimmerman_electronnum = (zimmerman_charge/1.60217663e-19)*voxel_area
print(f"will take {zimmerman_electronnum/(np.max(ilm_values[ilm_values!=0])/4)} iterations at this rate to get to the photoemission flux ranges shown at 3 seconds")

In [ ]:
# 5 um 
configIN = "onlysolarwind"
filenames = sorted(glob.glob(f"../build-adaptive-mesh/fieldmaps/*{configIN}*.txt"))

for fileIN in filenames:

    print(fileIN)
    iterationIN = int(fileIN.split("-")[-2]) #-50

    if iterationIN > 0:

        # Usage
        vars()["field_"+str(iterationIN)],_ = read_adaptive_fieldmap(fileIN)
        vars()["field_"+str(iterationIN)]["E_mag"]=np.linalg.norm(vars()["field_"+str(iterationIN)][["Ex", "Ey", "Ez"]].values, axis=1)

        # plot the distribution of the field to get a sense for the overall field values before analyzing the field in detail
        plt.hist(vars()["field_"+str(iterationIN)]["E_mag"],bins=np.logspace(0,10,100))
        plt.xscale("log")
        plt.yscale("log")
        plt.xlabel("|E| (V/m)")
        plt.ylabel("Counts")
        plt.title("field_"+str(iterationIN))
        plt.show()
    
    print(72*"-")

In [ ]:
stacked_spheres_cropped = stacked_spheres.copy()

bbox_min = np.array([-0.2, -0.3, 0])
bbox_max = np.array([ 0.2,  0.1,  100])

# Filter vertices
in_box = np.all((stacked_spheres_cropped.vertices >= bbox_min) & 
                (stacked_spheres_cropped.vertices <= bbox_max), axis=1)

# Get face indices where all 3 vertices are in the box
face_mask = np.all(in_box[stacked_spheres_cropped.faces], axis=1)

# Extract submesh
stacked_spheres_cropped = stacked_spheres_cropped.submesh([face_mask], only_watertight=False, append=True)

# Usage with black semi-transparent edges
scene = plot_trimesh_edges_only(stacked_spheres_cropped, edge_color=[0, 0, 0, 350])  # Black, 50% transparent

# Create a PointCloud object
#gamma_photoemission_sites = trimesh.points.PointCloud(np.array(all_gamma_holes_df["Pre_Step_Position_mm"].tolist()), colors=[0, 255, 0, 255])  # RGBA green points
iterationIN = 2
fieldIN = vars()["field_"+str(iterationIN)]
largefieldpoints = np.array(fieldIN[fieldIN["E_mag"] > 1e-5][["x", "y", "z"]].to_numpy().tolist())

# fieldpoints = []

# for pointIN in largefieldpoints:
#     sphere = trimesh.creation.icosphere(radius=0.005)  # Create a sphere for each point
    
#     # Apply translation to the sphere (move it to the position of the point)
#     sphere.apply_translation(np.array(pointIN))
    
#     # Set the sphere color to red
#     sphere.visual.face_colors = [255, 0, 0, 255]  # red spheres
    
#     # Append the sphere to the list
#     fieldpoints.append(sphere)

# # Add all the spheres to the scene
# scene.add_geometry(fieldpoints)

max_arrows = 3000
field_nonzero = fieldIN[fieldIN["E_mag"] > 1e-7]
field_plot = field_nonzero.sample(n=max_arrows, random_state=42) if len(field_nonzero) > max_arrows else field_nonzero

directions = field_plot[['Ex', 'Ey', 'Ez']].values
magnitudes = np.linalg.norm(directions, axis=1)
directions_unit = np.where(magnitudes[:, None] > 0, directions / magnitudes[:, None], 0)

# Avoid log(0) by adding a small value (e.g., 1e-12) if needed
log_magnitudes = np.log10(magnitudes + 1e-12)  # Log base 10 normalization

# Normalize log-magnitudes to [0, 1]
norm_magnitudes = (log_magnitudes - log_magnitudes.min()) / (np.ptp(log_magnitudes) + 1e-12)

# Create a colormap
cmap = plt.cm.jet
colors_rgba = cmap(norm_magnitudes)  # shape: (N, 4) with floats in [0, 1]

# Convert to 0–255 uint8 for trimesh
colors_rgba = (colors_rgba * 255).astype(np.uint8)

# Define base dimensions
base_arrow_length = 0.05  # This will be scaled by magnitude
arrow_radius = 0.001
cone_ratio = 0.2  # Cone length as fraction of total arrow length

# Normalize magnitudes for scaling arrow lengths
scaling_factor = 1.0  # Adjust this to control overall arrow size
scaled_lengths = base_arrow_length * scaling_factor * norm_magnitudes

for pos, dir_vec, color, magnitude_norm, scaled_length in zip(
    field_plot[['x', 'y', 'z']].values, 
    directions_unit, 
    colors_rgba, 
    norm_magnitudes,
    scaled_lengths
):
    if np.linalg.norm(dir_vec) == 0:
        continue

    # Calculate arrow dimensions based on scaled length
    arrow_length = scaled_length
    cone_length = arrow_length * cone_ratio
    shaft_length = arrow_length - cone_length

    # Skip very small arrows to avoid visualization issues
    # if arrow_length < 1e-4:
    #     continue

    # Create shaft - centered at origin along Z-axis
    shaft = trimesh.creation.cylinder(radius=arrow_radius, height=shaft_length, sections=12)
    shaft.apply_translation([0, 0, shaft_length / 2])  # Move so base is at z=0, top at z=shaft_length

    # Create cone - positioned to touch the shaft
    cone = trimesh.creation.cone(radius=arrow_radius * 2, height=cone_length, sections=12)
    # Position cone so its base touches the top of the shaft (at z=shaft_length)
    cone.apply_translation([0, 0, shaft_length])

    # Combine parts
    arrow = trimesh.util.concatenate([shaft, cone])

    # Set color per face
    arrow.visual.face_colors = np.tile(color, (arrow.faces.shape[0], 1))

    # Align +Z to direction
    transform = trimesh.geometry.align_vectors([0, 0, 1], dir_vec)
    transform[:3, 3] = pos
    arrow.apply_transform(transform)

    scene.add_geometry(arrow)

scene.show()

In [ ]:
# Target point
target = np.array([0.03, -0.1, 0.227]) 
#target = np.array([0, -0.1, 0.20700000000000002])

# DataFrame to store results
efield_values = []

# Iterate over field_1, field_2, etc.
for iterationIN in range(101, 118):
    fieldIN = vars()["field_" + str(iterationIN)]  # Field data for the current iteration

    # Compute Euclidean distance to all points in the field
    distances = np.linalg.norm(fieldIN[['x', 'y', 'z']].values - target, axis=1)

    # Find index of the closest point
    closest_idx = np.argmin(distances)

    # Extract the closest point and its electric field
    closest_idx = np.argmax(fieldIN["E_mag"])
    closest_point = fieldIN.iloc[closest_idx].copy()  # Explicitly make a copy to avoid SettingWithCopyWarning

    # print(f"Closest point for iteration {iterationIN}:")
    # print(closest_point[['x', 'y', 'z']])
    # print("\nElectric field at that point:")
    # print(closest_point[['Ex', 'Ey', 'Ez']])
    
    # Add iteration number
    closest_point["iter"] = iterationIN

    # Append the result to the efield_values list
    efield_values.append(closest_point[['iter', 'Ex', 'Ey', 'Ez']])

# Create a new DataFrame from the efield_values list
efield_df = pd.DataFrame(efield_values)


In [ ]:
# Perform linear fitting (degree 1 for linear)
mag_values = efield_df["Ex"] #np.linalg.norm(efield_df[["Ex", "Ey", "Ez"]].values, axis=1) #efield_df["Ex"]

# Plot the original data
plt.plot(efield_df["iter"]-100, mag_values, '.-', label="Original Data")

# Labeling the plot
plt.xlabel("Iteration")
plt.ylabel(r"E$_z$ (V/m)")
plt.ylabel(r"|E| (V/m)")
plt.legend()
plt.show()

# Print the extrapolated value at x=50#print(f"Extrapolated value at x = 50: {extrapolated_value:.2f} V/m")


In [ ]:
mag_values

In [ ]:
np.arraymag_values[0]/1e5

In [ ]:
94209+81883

In [ ]:
136752+155429

After iteration 1 (total e-: 1139228, Holes:     870236)

------------------------------------------
After iteration 2 (total e-: 8,660)

Closest point:
x    0.000000
y   -0.100000
z    0.206795
Name: 38481, dtype: float64

Electric field at that point:
Ex    1.360211e-07
Ey    1.542312e-08
Ez    8.020124e-08
Name: 38481, dtype: float64

------------------------------------------
After iteration 1 (total e-: 4,356)

Closest point:
x    0.000000
y   -0.100000
z    0.206795
Name: 4605855, dtype: float64

Electric field at that point:
Ex    8.230766e-08
Ey   -5.993845e-09
Ez    4.336823e-08
Name: 4605855, dtype: float64

In [ ]:
# define the size of the world
WorldX, WorldY, WorldZ = 200, 600, 606.61

# think we need to get to ~8 iterations at 1 million particles each to get to 3s 
((4*1e-6*(WorldX*WorldY)/(1e6**2))*6.241509*10**18*3)/1e6

In [ ]:
1/1.60217663e-19

In [ ]:
voxel_area = 0.0004/(1000)**2 # rough area approximated from python (0.4 micron2)
zimmerman_charge = 0.5*(1e-6) # C/m2
(zimmerman_charge/1.60217663e-19)*voxel_area


In [ ]:
voxel_area

In [ ]:
385*1e-3

In [ ]:
voxel_area

In [ ]:
print("up through iteration 33")
surface,pot = plot_surface_potential_fornegativepositive_charge(all_electrons_inside_df, all_gamma_holes_df, stacked_spheres, vmin=-0.2,vmax=0.2)
print(min(pot),max(pot))
# Make sure each triangle has its own unique vertices
surface_edited = surface.copy()
surface_edited.unmerge_vertices()
surface_edited.visual.vertex_colors = None
surface_edited.show()

In [ ]:
bbox_min = np.array([-0.2, -0.3, 0])
bbox_max = np.array([ 0.2,  0.1,  100])

# Filter vertices
in_box = np.all((surface_edited.vertices >= bbox_min) & 
                (surface_edited.vertices <= bbox_max), axis=1)

# Get face indices where all 3 vertices are in the box
face_mask = np.all(in_box[surface_edited.faces], axis=1)

# Extract submesh
cropped = surface_edited.submesh([face_mask], only_watertight=False, append=True)
cropped.show()

In [ ]:
# Define colormap and normalization
cmap = plt.cm.seismic
norm = Normalize(vmin=-0.2, vmax=0.2)

# Create a figure and a single axis for the colorbar
fig, ax = plt.subplots(figsize=(4, 0.5))
fig.subplots_adjust(bottom=0.5)

# Create the colorbar
cb = ColorbarBase(ax, cmap=cmap, norm=norm, orientation='horizontal')
cb.set_label('Surface Potential (mV)')  # Optional label

plt.show()


## Case 3: all particles (incident e-, protons, gammas)

In [ ]:
directory_path = "../build-sphere-charging/root/"
configIN = "allparticles"
filelist = sorted(glob.glob(f"{directory_path}/*stackediteration*{configIN}*num5000.root"))

all_gamma_holes,all_photoemission_electrons,all_protons_inside,all_electrons_inside = [],[],[],[]

for fileIN in filelist:
    print(fileIN.split("/")[-1])
    number_str = fileIN.split("/")[-1].split("_")[1]
    iterationNUM = int(''.join(filter(str.isdigit, number_str)))

    # read data from different iterations
    vars()["gamma_holes_"+str(number_str)], vars()["photoemission_electrons_inside_"+str(number_str)], \
        vars()["protons_inside_"+str(number_str)], vars()["electrons_inside_"+str(number_str)] = calculate_stats(read_rootfile(fileIN.split("/")[-1], directory_path=directory_path), \
                                                                                                             config=configIN)
    all_gamma_holes.append(vars()["gamma_holes_"+str(number_str)])
    all_photoemission_electrons.append(vars()["photoemission_electrons_inside_"+str(number_str)])
    all_protons_inside.append(vars()["protons_inside_"+str(number_str)])
    all_electrons_inside.append(vars()["electrons_inside_"+str(number_str)])
    print(78*"-")

# Concatenate all iterations into single DataFrames
all_gamma_holes_df = pd.concat(all_gamma_holes, ignore_index=True)
all_photoemission_electrons_df = pd.concat(all_photoemission_electrons, ignore_index=True)
all_protons_inside_df = pd.concat(all_protons_inside, ignore_index=True)
all_electrons_inside_df = pd.concat(all_electrons_inside, ignore_index=True)

In [ ]:
# Concatenate all iterations into single DataFrames
all_gamma_holes_df = pd.concat(all_gamma_holes, ignore_index=True)
all_photoemission_electrons_df = pd.concat(all_photoemission_electrons, ignore_index=True)
all_protons_inside_df = pd.concat(all_protons_inside, ignore_index=True)
all_electrons_inside_df = pd.concat(all_electrons_inside, ignore_index=True)

In [ ]:
print("up through iteration 28")
# input order: gammas, photoelectrons, protons, electrons, convex_combined,
surface,facecolors = plot_surface_potential_allparticle_case(all_gamma_holes_df, all_photoemission_electrons_df, all_protons_inside_df, all_electrons_inside_df, 
                                                      stacked_spheres, vmin=-0.2,vmax=0.2)
print(min(facecolors),max(facecolors))
plt.plot(facecolors[facecolors!=0])
plt.show()

# Make sure each triangle has its own unique vertices
surface_edited = surface.copy()
surface_edited.unmerge_vertices()
surface_edited.visual.vertex_colors = None
surface_edited.show()

In [ ]:
bbox_min = np.array([-0.2, -0.3, 0])
bbox_max = np.array([ 0.2,  0.1,  100])

# Filter vertices
in_box = np.all((surface_edited.vertices >= bbox_min) & 
                (surface_edited.vertices <= bbox_max), axis=1)

# Get face indices where all 3 vertices are in the box
face_mask = np.all(in_box[surface_edited.faces], axis=1)

# Extract submesh
cropped = surface_edited.submesh([face_mask], only_watertight=False, append=True)
cropped.show()

In [ ]:
# Define colormap and normalization
cmap = plt.cm.seismic
norm = Normalize(vmin=-0.2, vmax=0.2)

# Create a figure and a single axis for the colorbar
fig, ax = plt.subplots(figsize=(4, 0.5))
fig.subplots_adjust(bottom=0.5)

# Create the colorbar
cb = ColorbarBase(ax, cmap=cmap, norm=norm, orientation='horizontal')
cb.set_label('Surface Potential (mV)')  # Optional label

plt.show()


In [ ]:
print("up through iteration 19: only SW ions")
surface,facecolors = plot_surface_potential_fornegativepositive_charge(all_electrons_inside_df, all_protons_inside_df, stacked_spheres, vmin=-0.2,vmax=0.2)

print(min(facecolors),max(facecolors))
plt.plot(facecolors[facecolors!=0])
plt.show()

# Make sure each triangle has its own unique vertices
surface_edited = surface.copy()
surface_edited.unmerge_vertices()
surface_edited.visual.vertex_colors = None
surface_edited.show()

In [ ]:
print("up through iteration 22: only photons")
surface,facecolors = plot_surface_potential_fornegativepositive_charge(all_photoemission_electrons_df, all_gamma_holes_df, stacked_spheres, vmin=-0.5,vmax=0.5)

print(min(facecolors),max(facecolors))
plt.plot(facecolors[facecolors!=0])
plt.show()

# Make sure each triangle has its own unique vertices
surface_edited = surface.copy()
surface_edited.unmerge_vertices()
surface_edited.visual.vertex_colors = None
surface_edited.show()

In [ ]:
bbox_min = np.array([-0.2, -0.3, 0])
bbox_max = np.array([ 0.2,  0.1,  100])

# Filter vertices
in_box = np.all((surface_edited.vertices >= bbox_min) & 
                (surface_edited.vertices <= bbox_max), axis=1)

# Get face indices where all 3 vertices are in the box
face_mask = np.all(in_box[surface_edited.faces], axis=1)

# Extract submesh
cropped = surface_edited.submesh([face_mask], only_watertight=False, append=True)
cropped.show()
